In [5]:
import matplotlib.pyplot as plt
   
from numpy import argmax
import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

InputFile ='/content/drive/MyDrive//Dataset/CheckEquality.xlsx'
Model_Path ='/content/drive/MyDrive/Dataset/28_4_LastModel'
OutputFile ='/content/drive/MyDrive/Dataset/Predicted_Trend'+datetime.now().strftime("%d_%m_%Y_%H_%M")+'.xlsx'

def find_MaxtimeStep(df):
    # x = df.loc[df['INDEX'].notnull()].index.tolist()
    # max_distance =0 
    # for i in range (0,len(x)-1):
    #     if x[i+1]-x[i] >max_distance:
    #         max_distance = x[i+1]-x[i]
    return 700
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles

def PrepareTestData(df,Max_TimeStep):   
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'INDEX'])):
            y.append(df.at[i,'INDEX'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
            for j in range (0,4):
                candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
                #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    return np.array(x),np.array(y)

excel_test_data =  pd.read_excel(InputFile)
df1 = pd.DataFrame(excel_test_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND','INDEX'])
Max_TimeStep = find_MaxtimeStep(df1)
testX,textY = PrepareTestData(df1,Max_TimeStep)
scaler = StandardScaler()
pd.DataFrame(testX[0]).to_excel('/content/drive/MyDrive//Dataset/TestEquality_BeforeNormalize.xlsx')
for i in range(0,testX.shape[0]):
   testX[i]= scaler.fit_transform(testX[i])
pd.DataFrame(testX[0]).to_excel('/content/drive/MyDrive//Dataset/TestEquality_Afternormalize.xlsx')
model = keras.models.load_model(Model_Path)
testPredict = model.predict(testX)
df1[['BreakDown','BreakUp','Channel','Down','Side','Up']] = pd.DataFrame(testPredict)

trendlist = df1.loc[df1['INDEX'].notnull()].index.tolist()
answer = ["" for i in range(len(df1))]
BreakDown = ["" for i in range(len(df1))]
BreakUp = ["" for i in range(len(df1))]
Channel = ["" for i in range(len(df1))]
Down = ["" for i in range(len(df1))]
Side = ["" for i in range(len(df1))]
Up = ["" for i in range(len(df1))]
key=['BreakDown','BreakUp','Channel','Down','Side','Up']
for i in range(0,len(trendlist)):
  answer[trendlist[i]] =key[argmax(testPredict[i, :])]
  BreakDown[trendlist[i]] =testPredict[i, 0]
  BreakUp[trendlist[i]] =testPredict[i, 1]
  Channel[trendlist[i]] =testPredict[i, 2]
  Down[trendlist[i]] =testPredict[i, 3]
  Side[trendlist[i]] =testPredict[i, 4]
  Up[trendlist[i]] =testPredict[i, 5]
df1['BreakDown'] = BreakDown
df1['BreakUp'] = BreakUp
df1['Channel'] = Channel
df1['PredictedTrend'] = answer
df1['Down'] = Down
df1['Side'] = Side
df1['Up'] = Up
df1.to_excel(OutputFile)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')